In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime
from datetime import datetime, date

In [19]:
# Настройка вывода данных
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

## Сбор данных

In [20]:
df_f = pd.read_excel("d.xlsx", sheet_name='Лист1')
df_f

,Территориальная иерархия ДНС,Абк в ТЦ Калина ФТ,Unnamed: 2,Unnamed: 3,Unnamed: 4,...,Unnamed: 3385,Unnamed: 3386,Unnamed: 3387,Unnamed: 3388,Unnamed: 3389
0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1,Сумма выручки,Названия столбцов,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,Названия строк,2011-08-16,2011-09-05,2011-09-06,2011-09-07,...,2020-12-16,2020-12-17,2020-12-18,2020-12-19,Общий итог
3,01. Ноутбуки,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
4,01. Нетбуки,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2442615
...,...,...,...,...,...,...,...,...,...,...,...
267,Цепные пилы,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,358979
268,Шуруповерты,NaN,NaN,NaN,NaN,...,NaN,2250,NaN,7999,1.45469e+06
269,Отсутствует,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
270,Отсутствует,0,99012,107181,271470,...,1703837,1376023,2155250,2.57066e+06,1.54422e+09


## Подготовка данных

In [21]:
# Удаление лишних столбцов и колонок
df_clear = df_f.drop(index=[0, 1, 269, 270, 271], columns='Unnamed: 3389').reset_index().drop(columns='index')

In [22]:
# Транспонирование таблицы из строк в столбцы
df_test = df_clear.transpose().reset_index(drop=True)
df_test

,0,1,2,3,4,...,262,263,264,265,266
0,Названия строк,01. Ноутбуки,01. Нетбуки,02. Ноутбуки,03. Аксессуары для ноутбуков,...,Перфораторы,Сварочные Аппараты,Тепловые пушки,Цепные пилы,Шуруповерты
1,2011-08-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,2011-09-05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
3,2011-09-06,NaN,NaN,65797,NaN,...,NaN,NaN,NaN,NaN,NaN
4,2011-09-07,NaN,NaN,86395,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3384,2020-12-15,NaN,NaN,362697,-2499,...,NaN,NaN,NaN,NaN,NaN
3385,2020-12-16,NaN,NaN,380990,NaN,...,14158,NaN,NaN,NaN,NaN
3386,2020-12-17,NaN,NaN,134997,2150,...,NaN,NaN,NaN,NaN,2250
3387,2020-12-18,NaN,NaN,181446,NaN,...,NaN,NaN,NaN,NaN,NaN


In [23]:
# Применение к строке из списка роли имен колонок.
df_test.columns = df_test.iloc[0]

fragment_01 = df_test.reindex(df_test.index.drop(1)).drop(index=0)
fragment_01 = fragment_01.rename(columns={"Названия строк": 'Дата'}).reset_index(drop=True)
fragment_01.columns.name = 'ID'

In [24]:
# Получить все имена колонок.
fragment_01.columns.tolist()[:5]

['Дата',
 '01. Ноутбуки',
 '01. Нетбуки',
 '02. Ноутбуки',
 '03. Аксессуары для ноутбуков']

In [25]:
# Удаление ненужных колонок
fragment_01 = fragment_01.drop(columns=[
    '01. Ноутбуки',
    '02. Компьютеры',
    'EOL/Не используется',
    '03. Комплектующие',
    '04. Мониторы',
    '05. Сотовые, планшеты и электронные книги',
    '06. Камеры цифровые',
    '07. Карманные гаджеты',
    '08. ТВ и медиа',
    '09. Акустика',
    '10. Кабельная продукция',
    '11. Печатающая офисная техника/Оргтехника',
    '13. Источники бесперебойного питания',
    '14. Мелочи и аксессуары',
    '16. Чехлы и сумки',
    '17. Расходники и накопители информации',
    '18. Бытовая техника',
    '19. Автоэлектроника',
    '20. Интертеймент',
    '21. Сетевое Оборудование',
    '22. Источники питания',
    '23. Сумки',
    '26. Инструменты'
])

In [26]:
# Общее кол-во пропущенных элементов
# fragment_01.isnull().sum().sum()

In [27]:
# Приведение типа "datetime64" к колонке.
fragment_01['Дата'] = pd.to_datetime(fragment_01['Дата'], format='%Y-%m-%d')

In [37]:
fragment_01 # Как же бесит то, что без регулярных выражений эти (01. Нетбуки	02. Ноутбуки...) не убрать. Данные полученные из Excel(((.

ID,Дата,01. Нетбуки,02. Ноутбуки,03. Аксессуары для ноутбуков,ГСБ,...,Перфораторы,Сварочные Аппараты,Тепловые пушки,Цепные пилы,Шуруповерты
0,2011-09-05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1,2011-09-06,NaN,65797,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,2011-09-07,NaN,86395,NaN,6650,...,NaN,NaN,NaN,NaN,NaN
3,2011-09-08,27954,48279,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
4,2011-09-09,NaN,56004,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3382,2020-12-15,NaN,362697,-2499,51998,...,NaN,NaN,NaN,NaN,NaN
3383,2020-12-16,NaN,380990,NaN,NaN,...,14158,NaN,NaN,NaN,NaN
3384,2020-12-17,NaN,134997,2150,45048,...,NaN,NaN,NaN,NaN,2250
3385,2020-12-18,NaN,181446,NaN,55198,...,NaN,NaN,NaN,NaN,NaN


## Подготовка формул прогноза.